В файле <i>orders</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта, 
    price - цена единицы товара, 
    quantity - количество единиц товара в заказе
```
В файле <i>categories</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта
```

## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

* Определите, какого числа было сделано больше всего заказов?
* Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)
* Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

In [17]:
# %pip install pandas

In [18]:
import pandas as pd

In [19]:
categories = pd.read_pickle('data/categories.pkl')
orders = pd.read_pickle('data/orders.pkl')

In [20]:
categories

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты
...,...,...,...
35025,999942,Шоколадные батончики,Сладости
35026,999964,Зелень и салаты,Овощи и фрукты
35027,999965,"Пироги, сдоба, кексы, рулеты",Мучные изделия
35028,999970,"Прокладки, тампоны","Красота, гигиена, здоровье"


In [21]:
data = []

for i in orders['items']:
    data += i
    # c+=1
    # if c == 3:
    #     break
# print(data)
df_items = pd.DataFrame(data)
print(df_items)

        ItemID   price  quantity
0       413736   36.72         2
1       523171   90.32         1
2       452831  223.11         1
3       303673   59.90         7
4       318193  319.91         2
...        ...     ...       ...
852813  439036   17.50         1
852814  303765    6.56         1
852815  341794   69.90         1
852816  307216    2.99         1
852817  322455    9.68         1

[852818 rows x 3 columns]


In [22]:
# Вычисляем price * quantity и добавляем в новый DataFrame
# пример [{'ItemID': '306947', 'price': 89.9, 'quantity': 2}]
result_df = pd.DataFrame({
    'ItemID': df_items['ItemID'],
    'Total Price': df_items['price'] * df_items['quantity']

})
print(result_df)

        ItemID  Total Price
0       413736        73.44
1       523171        90.32
2       452831       223.11
3       303673       419.30
4       318193       639.82
...        ...          ...
852813  439036        17.50
852814  303765         6.56
852815  341794        69.90
852816  307216         2.99
852817  322455         9.68

[852818 rows x 2 columns]


In [23]:
# # Суммируем Total Price по ItemID
summed_df = result_df.groupby('ItemID', as_index=False).sum()
print(summed_df)

        ItemID  Total Price
0      1000002       199.30
1      1000046      3810.38
2      1000048       191.40
3      1000049       898.52
4      1000061       341.81
...        ...          ...
35025   999942       294.60
35026   999964       159.90
35027   999965       665.50
35028   999970      3436.81
35029   999971      9508.30

[35030 rows x 2 columns]


In [24]:
# # Делаем DataFrame categories из двух столбцов
categories = categories[['ItemID', 'ParentCategory']]
print(categories)

        ItemID              ParentCategory
0      1000002                    Сладости
1      1000046                     Напитки
2      1000048                    Сладости
3      1000049                     Напитки
4      1000061              Овощи и фрукты
...        ...                         ...
35025   999942                    Сладости
35026   999964              Овощи и фрукты
35027   999965              Мучные изделия
35028   999970  Красота, гигиена, здоровье
35029   999971          Молочная продукция

[35030 rows x 2 columns]


In [25]:
final_df = pd.merge(summed_df, categories, on='ItemID')
print(final_df)

        ItemID  Total Price              ParentCategory
0      1000002       199.30                    Сладости
1      1000046      3810.38                     Напитки
2      1000048       191.40                    Сладости
3      1000049       898.52                     Напитки
4      1000061       341.81              Овощи и фрукты
...        ...          ...                         ...
35025   999942       294.60                    Сладости
35026   999964       159.90              Овощи и фрукты
35027   999965       665.50              Мучные изделия
35028   999970      3436.81  Красота, гигиена, здоровье
35029   999971      9508.30          Молочная продукция

[35030 rows x 3 columns]


In [26]:
group_parentcategory = final_df.groupby('ParentCategory', as_index=False).sum()
print(group_parentcategory)

                ParentCategory  \
0                      Бакалея   
1                Бытовая химия   
2                    Для детей   
3              Для дома и сада   
4                Для праздника   
5                    Для школы   
6                    Зоотовары   
7   Красота, гигиена, здоровье   
8      Кухонные принадлежности   
9           Молочная продукция   
10              Мучные изделия   
11           Мясо, птица, рыба   
12                     Напитки   
13              Овощи и фрукты   
14          Системы нагревания   
15                    Сладости   
16                    Текстиль   

                                               ItemID  Total Price  
0   1000095100009710001031000114100017010001771000...  25223205.54  
1   1000479100048610005061000531100053210011181001...   6690247.78  
2   1000766100076710007681000829100084010008731000...   2684441.54  
3   1000366100037210003791000384100038910003961000...   1706359.70  
4   10000851000316100032010003261000340100

In [27]:
# Сортируем по Total Price от большего к меньшему
group_parentcategory = group_parentcategory.sort_values(by='Total Price', ascending=False)
print(group_parentcategory)

                ParentCategory  \
11           Мясо, птица, рыба   
9           Молочная продукция   
13              Овощи и фрукты   
0                      Бакалея   
12                     Напитки   
7   Красота, гигиена, здоровье   
10              Мучные изделия   
1                Бытовая химия   
15                    Сладости   
6                    Зоотовары   
2                    Для детей   
3              Для дома и сада   
8      Кухонные принадлежности   
4                Для праздника   
14          Системы нагревания   
16                    Текстиль   
5                    Для школы   

                                               ItemID  Total Price  
11  1000215100022210002541000259100046110004671001...  32268493.46  
9   1000105100041810005391000646100076210007631000...  27702995.43  
13  1000061100006210000641000066100011610001171000...  27607486.90  
0   1000095100009710001031000114100017010001771000...  25223205.54  
12  10000461000049100011510002211000237100

In [28]:
# Выводим результат
print(group_parentcategory[['ParentCategory', 'Total Price']].head(3))

        ParentCategory  Total Price
11   Мясо, птица, рыба  32268493.46
9   Молочная продукция  27702995.43
13      Овощи и фрукты  27607486.90
